In [6]:
import { authenticate } from './opencrvs-api.ts'
const token = await authenticate('k.mweene', 'test');

import api from "npm:@opencrvs/toolkit@1.9.0-rc.f45cc23/api";
const { createClient } = api;
const client = createClient('http://localhost:7070', `Bearer ${token}`);
